In [1]:
import os
file_names:list[str] = os.listdir('stations')
abs_paths:list[str] = []
for name in file_names:
    if '每日各站進出站人數' in name:
        abs_paths.append(os.path.abspath(f'stations/{name}'))
    
abs_paths

['/workspaces/2024-2-22/ lesson19/stations/每日各站進出站人數2020.csv',
 '/workspaces/2024-2-22/ lesson19/stations/每日各站進出站人數20190423-20191231.csv',
 '/workspaces/2024-2-22/ lesson19/stations/每日各站進出站人數2023.csv',
 '/workspaces/2024-2-22/ lesson19/stations/每日各站進出站人數2021.csv',
 '/workspaces/2024-2-22/ lesson19/stations/每日各站進出站人數2022.csv']

In [2]:
import pandas as pd
from pandas import DataFrame

all_df:list[DataFrame] = [pd.read_csv(path) for path in abs_paths]
display(all_df)


[       trnOpDate  staCode  gateInComingCnt  gateOutGoingCnt
 0       20200101      900             7552             8154
 1       20200101      910             1020             1135
 2       20200101      920             1623             1972
 3       20200101      930             4130             4813
 4       20200101      940             1818             2128
 ...          ...      ...              ...              ...
 86927   20201231     7360             3523             3602
 86928   20201231     7361                3               18
 86929   20201231     7362               28               34
 86930   20201231     7380              609              604
 86931   20201231     7390              377              444
 
 [86932 rows x 4 columns],
        trnOpDate  staCode  gateInComingCnt  gateOutGoingCnt
 0       20190423      900             8442             7743
 1       20190423      910             1394             1348
 2       20190423      920             2770             

In [8]:
inOut_df = pd.concat(all_df)
inOut_df.columns = ['乘車日','車站代碼','進站人數','出站人數']
inOut_df

,乘車日,車站代碼,進站人數,出站人數
0,20200101,900,7552,8154
1,20200101,910,1020,1135
2,20200101,920,1623,1972
3,20200101,930,4130,4813
4,20200101,940,1818,2128
...,...,...,...,...
86547,20221231,7360,3984,3893
86548,20221231,7361,92,115
86549,20221231,7362,131,159
86550,20221231,7380,494,426


In [10]:
import requests
url2 = 'https://ods.railway.gov.tw/tra-ods-web/ods/download/dataResource/0518b833e8964d53bfea3f7691aea0ee'
response = requests.get(url2)
if response.status_code == 200:
    station_data = response.json()

station_df = pd.DataFrame(station_data,columns=['stationCode','stationName'])
station_df.columns = ['車站代碼','中文站名']
station_df['車站代碼'] = station_df['車站代碼'].astype(int)
station_df

,車站代碼,中文站名
0,900,基隆
1,910,三坑
2,920,八堵
3,930,七堵
4,940,百福
...,...,...
237,7350,猴硐
238,7360,瑞芳
239,7361,海科館
240,7380,四腳亭


In [ ]:
station_all_df = pd.merge(inOut_df,station_df,on='車站代碼',how='left')
station_all_df

In [11]:
station_all_df.info()

,乘車日,車站代碼,進站人數,出站人數,中文站名
0,20200101,900,7552,8154,基隆
1,20200101,910,1020,1135,三坑
2,20200101,920,1623,1972,八堵
3,20200101,930,4130,4813,七堵
4,20200101,940,1818,2128,百福
...,...,...,...,...,...
406756,20221231,7360,3984,3893,瑞芳
406757,20221231,7361,92,115,海科館
406758,20221231,7362,131,159,NaN
406759,20221231,7380,494,426,四腳亭


In [12]:
#移除Na
station_all_df1 = station_all_df.dropna()
station_all_df1.info()


<class 'pandas.core.frame.DataFrame'>
Index: 405059 entries, 0 to 406760
Data columns (total 5 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   乘車日     405059 non-null  int64 
 1   車站代碼    405059 non-null  int64 
 2   進站人數    405059 non-null  int64 
 3   出站人數    405059 non-null  int64 
 4   中文站名    405059 non-null  object
dtypes: int64(4), object(1)
memory usage: 18.5+ MB


In [13]:
#乘車日變為datetime
station_all_df1['乘車日'] = pd.to_datetime(station_all_df1['乘車日'].astype(str))
station_all_df1.info()

<class 'pandas.core.frame.DataFrame'>
Index: 405059 entries, 0 to 406760
Data columns (total 5 columns):
 #   Column  Non-Null Count   Dtype         
---  ------  --------------   -----         
 0   乘車日     405059 non-null  datetime64[ns]
 1   車站代碼    405059 non-null  int64         
 2   進站人數    405059 non-null  int64         
 3   出站人數    405059 non-null  int64         
 4   中文站名    405059 non-null  object        
dtypes: datetime64[ns](1), int64(3), object(1)
memory usage: 18.5+ MB


/tmp/ipykernel_45775/2747619846.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  station_all_df1['乘車日'] = pd.to_datetime(station_all_df1['乘車日'].astype(str))


In [14]:
station_all_df2 = station_all_df1.sort_values(by='乘車日')
station_all_df2

,乘車日,車站代碼,進站人數,出站人數,中文站名
86932,2019-04-23,900,8442,7743,基隆
87084,2019-04-23,5070,533,424,南州
87085,2019-04-23,5080,15,14,鎮安
87086,2019-04-23,5090,368,325,林邊
87087,2019-04-23,5100,95,95,佳冬
...,...,...,...,...,...
233515,2023-12-31,3350,1229,3101,成功
233516,2023-12-31,3360,19617,16699,彰化
233517,2023-12-31,3370,1864,1532,花壇
233505,2023-12-31,3250,3517,3091,潭子


In [15]:
station_array = station_all_df2.values
station_array[:,0]

array([Timestamp('2019-04-23 00:00:00'), Timestamp('2019-04-23 00:00:00'),
       Timestamp('2019-04-23 00:00:00'), ...,
       Timestamp('2023-12-31 00:00:00'), Timestamp('2023-12-31 00:00:00'),
       Timestamp('2023-12-31 00:00:00')], dtype=object)

In [16]:
station_array[:,[0,2,3]]

array([[Timestamp('2019-04-23 00:00:00'), 8442, 7743],
       [Timestamp('2019-04-23 00:00:00'), 533, 424],
       [Timestamp('2019-04-23 00:00:00'), 15, 14],
       ...,
       [Timestamp('2023-12-31 00:00:00'), 1864, 1532],
       [Timestamp('2023-12-31 00:00:00'), 3517, 3091],
       [Timestamp('2023-12-31 00:00:00'), 1526, 656]], dtype=object)

In [17]:
station_all_df2.query('乘車日=="2023"')

/tmp/ipykernel_45775/2084214848.py:1: FutureWarning: The behavior of 'isin' with dtype=datetime64[ns] and castable values (e.g. strings) is deprecated. In a future version, these will not be considered matching by isin. Explicitly cast to the appropriate dtype before calling isin instead.
  station_all_df2.query('乘車日=="2023"')


,乘車日,車站代碼,進站人數,出站人數,中文站名
147237,2023-01-01,6090,4,6,東竹
147212,2023-01-01,5060,38,35,崁頂
147213,2023-01-01,5070,344,422,南州
147214,2023-01-01,5080,18,25,鎮安
147215,2023-01-01,5090,546,545,林邊
...,...,...,...,...,...
147140,2023-01-01,3310,1434,1327,五權
147139,2023-01-01,3300,37255,33837,臺中
147138,2023-01-01,3290,1222,1224,精武
147137,2023-01-01,3280,2145,2162,太原
